In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import os
from PIL import Image

# Define the image directory
image_directory = '../testing-images/Unseen'

# Define class names
class_names = ['Bean', 'Bitter_Gourd', 'Bottle_Gourd', 'Brinjal', 'Broccoli', 'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Cucumber', 'Papaya', 'Potato', 'Pumpkin', 'Radish', 'Tomato']

VGG_types = {
    "VGG16": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        "M",
    ],
}

class VGG_net(nn.Module):
    def __init__(self, input_channels, num_classes=2):
        super(VGG_net, self).__init__()
        self.in_channels = input_channels
        self.conv_layers = self.create_conv_layers(VGG_types['VGG16'])  # create our conv layers
        self.fcs = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes)  # sizeInputImage = 224, divided by num Maxpool : 224 / 2⁷ = 7
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)  # flatten our convlayers
        x = self.fcs(x)
        return x

    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels
        for layer in architecture:
            if type(layer) is int:
                out_channels = layer
                layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                    kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                            nn.BatchNorm2d(layer),
                            nn.ReLU()]
                in_channels = layer  # for the next iteration
            elif layer == 'M':
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]
        return nn.Sequential(*layers,)


# Load the trained model
model_path = '../fun2/VGG16_torch_model_epoch40.pt'
model = VGG_net(input_channels=3, num_classes=len(class_names))
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()

# Define the transformation for input images
data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# Function to predict and return the results along with raw predictions
def predict_image(image_path, model, class_names):
    try:
        image = Image.open(image_path)
        image_tensor = data_transform(image).unsqueeze(0)

        # Check if GPU is available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Move the input tensor to the appropriate device
        image_tensor = image_tensor.to(device)

        # Perform inference
        with torch.no_grad():
            model.eval()
            output = model(image_tensor)

        _, predicted_class = torch.max(output, 1)

        return {
            'image_path': image_path,
            'predicted_class': class_names[predicted_class.item()],
            'raw_predictions': output.squeeze().tolist()
        }

    except Exception as e:
        return {
            'image_path': image_path,
            'error': str(e)
        }

# Test the model on all files in the specified directory and print the results
for filename in os.listdir(image_directory):
    image_path = os.path.join(image_directory, filename)
    result = predict_image(image_path, model, class_names)
    print(result)

{'image_path': '../testing-images/Unseen\\1.jpg', 'predicted_class': 'Pumpkin', 'raw_predictions': [-1.4929288625717163, -1.2911391258239746, -0.23721100389957428, 1.0044753551483154, -1.2316761016845703, -2.0316882133483887, 2.5895533561706543, 0.10909018665552139, -0.5758165121078491, 2.5659568309783936, 1.8187466859817505, -0.9016873240470886, 3.058994770050049, -3.222970485687256, -0.3659025728702545]}
{'image_path': '../testing-images/Unseen\\2.jpg', 'predicted_class': 'Cabbage', 'raw_predictions': [-0.9408369660377502, -0.004756156355142593, -0.6222641468048096, 1.106011986732483, -0.018760809674859047, 2.407341241836548, -1.2085322141647339, -2.0814671516418457, 2.168921709060669, 0.04711754247546196, -0.4614936411380768, -0.9692177176475525, 1.4527355432510376, -1.2648720741271973, 0.5920839905738831]}
{'image_path': '../testing-images/Unseen\\3.jpg', 'predicted_class': 'Brinjal', 'raw_predictions': [2.0876855850219727, -0.6450721621513367, 0.02028268575668335, 3.09229826927185

In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import transforms
import os
from PIL import Image

# Define the image directory
image_directory = '../testing-images/Unseen'

# Define class names
class_names = ['Bean', 'Bitter_Gourd', 'Bottle_Gourd', 'Brinjal', 'Broccoli', 'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Cucumber', 'Papaya', 'Potato', 'Pumpkin', 'Radish', 'Tomato']

VGG_types = {
    "VGG16": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        "M",
    ],
}

class VGG_net(nn.Module):
    def __init__(self, input_channels, num_classes=2):
        super(VGG_net, self).__init__()
        self.in_channels = input_channels
        self.conv_layers = self.create_conv_layers(VGG_types['VGG16'])  # create our conv layers
        self.fcs = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes)  # sizeInputImage = 224, divided by num Maxpool : 224 / 2⁷ = 7
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)  # flatten our convlayers
        x = self.fcs(x)
        return x

    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels
        for layer in architecture:
            if type(layer) is int:
                out_channels = layer
                layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                    kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                            nn.BatchNorm2d(layer),
                            nn.ReLU()]
                in_channels = layer  # for the next iteration
            elif layer == 'M':
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]
        return nn.Sequential(*layers,)


# Load the trained model
model_path = '../fun2/VGG16_torch_model_epoch40.pt'
model = VGG_net(input_channels=3, num_classes=len(class_names))
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()

# Define the transformation for input images
data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# Function to predict and return the results along with raw predictions
def predict_image(image_path, model, class_names):
    try:
        image = Image.open(image_path)
        image_tensor = data_transform(image).unsqueeze(0)

        # Check if GPU is available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Move the input tensor to the appropriate device
        image_tensor = image_tensor.to(device)

        # Perform inference
        with torch.no_grad():
            model.eval()
            output = model(image_tensor)

        _, predicted_class = torch.max(output, 1)

        raw_predictions = output.squeeze().tolist()

        # Check if raw prediction for Cucumber is greater than 1.5
        if raw_predictions[class_names.index('Cucumber')] > 0.7:
            return {
                'image_path': image_path,
                'predicted_class': 'Cucumber',
            }
        else:
            return {
                'image_path': image_path,
                'predicted_class': class_names[predicted_class.item()],
            }

    except Exception as e:
        return {
            'image_path': image_path,
            'error': str(e)
        }

# Test the model on all files in the specified directory and print the results
for filename in os.listdir(image_directory):
    image_path = os.path.join(image_directory, filename)
    result = predict_image(image_path, model, class_names)
    print(result)

{'image_path': '../testing-images/Unseen\\1.jpg', 'predicted_class': 'Cucumber'}
{'image_path': '../testing-images/Unseen\\2.jpg', 'predicted_class': 'Cabbage'}
{'image_path': '../testing-images/Unseen\\3.jpg', 'predicted_class': 'Cucumber'}
{'image_path': '../testing-images/Unseen\\4.jpg', 'predicted_class': 'Cucumber'}
{'image_path': '../testing-images/Unseen\\5.jpg', 'predicted_class': 'Cucumber'}
{'image_path': '../testing-images/Unseen\\6.jpg', 'predicted_class': 'Tomato'}
{'image_path': '../testing-images/Unseen\\BEET-ROOT.jpg', 'predicted_class': 'Capsicum'}
{'image_path': '../testing-images/Unseen\\bottlegourd-approx-600-g-900-g-product.webp', 'predicted_class': 'Cucumber'}
{'image_path': '../testing-images/Unseen\\Bottle_Gourd.jpg', 'predicted_class': 'Cucumber'}
{'image_path': '../testing-images/Unseen\\brinjal-black-giant-round-28.webp', 'predicted_class': 'Potato'}
{'image_path': '../testing-images/Unseen\\brinjal_1_-Copy_2048x2048.webp', 'predicted_class': 'Capsicum'}
{'im